# Abigail's code try

In [1]:
!pip install wandb

In [1]:
import wandb
import numpy as np
import sys
import torch
import torch.utils.data as Data
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import pytorch_lightning as pl
from pytorch_lightning.loggers import WandbLogger
from pytorch_lightning.callbacks import LearningRateMonitor
from sklearn.metrics import r2_score
from scipy.stats import pearsonr
import torch.nn as nn

In [2]:
wandb.login()

wandb: Currently logged in as: amf2288 (fagerheim). Use `wandb login --relogin` to force relogin


True

In [3]:
BASE = 'models/'
PATH_NN= BASE+'NN_data_smooth/'

In [4]:
from funcs import regression_system
from funcs import fcnn
from funcs import dataset

In [5]:
# Define X,Y pairs (state, subgrid fluxes) for local network.local_torch_dataset = Data.TensorDataset(
BATCH_SIZE = 64  # Number of sample in each batch


In [6]:
submeso_dataset=dataset.SubmesoDataset(['grad_B','FCOR', 'Nsquared', 'HML', 'TAU',
              'Q', 'HBL', 'div', 'vort', 'strain'],res='1_4')

In [7]:
train_loader=DataLoader(
    submeso_dataset,
    #num_workers=1,
    batch_size=64,
    sampler=SubsetRandomSampler(submeso_dataset.train_ind))

In [8]:
test_loader=DataLoader(
    submeso_dataset,
    #num_workers=1,
    batch_size=len(submeso_dataset.test_ind),
    sampler=submeso_dataset.test_ind)


In [9]:
# use GPUs if available
if torch.cuda.is_available():
    print("CUDA Available")
    device = torch.device('cuda')
else:
    print('CUDA Not Available')
    device = torch.device('cpu')

CUDA Available


In [23]:
seed=123
batch_size=256
input_channels=10
output_channels=1
conv_layers = 2
kernel = 5
kernel_hidden = 3
activation="ReLU"
arch="fcnn"
epochs=100
save_path=BASE
save_name="test-3.pt"
lr=0.0025
wd=0.023133758465751404

In [24]:
## Wandb config file
config={"seed":seed,
        "lr":lr,
        "wd":wd,
        "batch_size":batch_size,
        "input_channels":input_channels,
        "output_channels":output_channels,
        "activation":activation,
        "save_name":save_name,
        "save_path":save_path,
        "arch":arch,
        "conv_layers":conv_layers,
        "kernel":kernel,
        "kernel_hidden":kernel_hidden,
        "epochs":epochs}

In [25]:
config["save_path"] + config["save_name"]

'models/test-3.pt'

In [26]:
wandb.init(project="submeso_ML",config=config)
model=fcnn.FCNN(config)
config["learnable parameters"]=sum(p.numel() for p in model.parameters())
system=regression_system.RegressionSystem(model,wandb.config["lr"],wandb.config["wd"])
wandb.watch(model, log_freq=1)
wandb_logger = WandbLogger()

cat: /sys/module/amdgpu/initstate: No such file or directory
ERROR:root:Driver not initialized (amdgpu not found in modules)


/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/loggers/wandb.py:397: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


In [27]:
trainer = pl.Trainer(
    default_root_dir=model.config["save_path"],
    accelerator="auto",
    max_epochs=config["epochs"],
    enable_progress_bar=False,
    logger=wandb_logger,
    )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [ ]:
trainer.fit(system, train_loader, test_loader)
#model.save_model()
torch.save(model, config["save_path"] + config["save_name"])

wandb.finish()
    

project_name="submeso_ML"

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type    | Params
--------------------------------------
0 | network   | FCNN    | 162 K 
1 | criterion | MSELoss | 0     
--------------------------------------
162 K     Trainable params
0         Non-trainable params
162 K     Total params
0.649     Total estimated model params size (MB)
/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/srv/conda/envs/notebook/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:438: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the va